# HybridIVPSolver — Adaptive-Step Solver Explained from Scratch

**Audience:** You know matrix operations. You do NOT need to know physics.  
**Prerequisite:** Read `hybrid_solver_explained.ipynb` first (covers mass matrices, Jacobians, KKT).

---

## How is this different from `HybridSolver`?

| Feature | HybridSolver | HybridIVPSolver |
|---------|-------------|----------------|
| Time step | **Fixed** — you choose `dt` | **Adaptive** — the algorithm chooses `dt` |
| Integration | Semi-implicit Euler (order 1) | Radau/BDF (order 3-5) |
| Accuracy | Low (needs small `dt`) | High (adjusts `dt` automatically) |
| Use case | Quick tests, simple systems | Stiff systems, production simulations |

### Why adaptive stepping?

- During **calm phases** (payload in steady descent), the solver takes **large steps** → fast
- During **violent phases** (parachute deployment, sudden forces), the solver takes **tiny steps** → accurate
- You just set accuracy tolerances (`rtol`, `atol`) and the solver figures out step sizes

### The mathematical formulation

Instead of stepping manually, we rewrite the problem as a standard ODE:

$$\frac{dy}{dt} = f(t, y)$$

where the state vector $y$ packs **everything** about every body:

$$y = [\underbrace{p_x, p_y, p_z}_{\text{position}}, \underbrace{q_x, q_y, q_z, q_w}_{\text{orientation}}, \underbrace{v_x, v_y, v_z}_{\text{velocity}}, \underbrace{\omega_x, \omega_y, \omega_z}_{\text{spin}}]_{\text{per body}}$$

That's **13 numbers per body**. Then `scipy.integrate.solve_ivp` handles the rest.

In [ ]:
import numpy as np
np.set_printoptions(precision=4, suppress=True, linewidth=120)

from aerislab.dynamics.body import RigidBody6DOF, quat_normalize, quat_derivative
from aerislab.dynamics.constraints import DistanceConstraint
from aerislab.core.solver import assemble_system, solve_kkt

## Setup: Same hanging-mass example

```
  ████ Anchor (1000 kg)   z = 2
    │  rod (2 m)
    ● Payload (5 kg)      z = 0
    ↓ gravity
```

In [ ]:
q_id = np.array([0., 0., 0., 1.])
g = np.array([0., 0., -9.81])

anchor = RigidBody6DOF("Anchor", 1000.0, np.diag([100.,100.,100.]),
                        np.array([0.,0.,2.]), q_id)
payload = RigidBody6DOF("Payload", 5.0, np.diag([0.5,0.5,0.5]),
                         np.array([0.,0.,0.]), q_id)
bodies = [anchor, payload]
constraint = DistanceConstraint(bodies, 0, 1, np.zeros(3), np.zeros(3), 2.0)

alpha, beta = 5.0, 1.0
print(f"Bodies: {[b.name for b in bodies]}")
print(f"State size per body: 13  (3 pos + 4 quat + 3 vel + 3 spin)")
print(f"Total state vector size: {13 * len(bodies)}")

---
# Step 1: State Packing — `_pack()`

We flatten all body states into one long vector that `scipy` can work with.

$$y = [\underbrace{p_0, q_0, v_0, \omega_0}_{\text{body 0 (13 values)}}, \underbrace{p_1, q_1, v_1, \omega_1}_{\text{body 1 (13 values)}}]$$

In [ ]:
def pack(bodies):
    """Pack all body states into a single flat array."""
    y = []
    for b in bodies:
        y.extend([*b.p, *b.q, *b.v, *b.w])  # 3 + 4 + 3 + 3 = 13
    return np.array(y, dtype=np.float64)

y0 = pack(bodies)
print(f"Packed state vector y0 (length {len(y0)}):")
for k, b in enumerate(bodies):
    off = 13 * k
    print(f"\n  {b.name} (indices {off}..{off+12}):")
    print(f"    position  y[{off}:{off+3}]   = {y0[off:off+3]}")
    print(f"    quaternion y[{off+3}:{off+7}]  = {y0[off+3:off+7]}")
    print(f"    velocity  y[{off+7}:{off+10}]  = {y0[off+7:off+10]}")
    print(f"    spin      y[{off+10}:{off+13}] = {y0[off+10:off+13]}")

# Step 2: Unpacking — `_unpack_to_world()`

The reverse: take a flat vector and write values back into each body object.

In [ ]:
def unpack_to_world(y, bodies):
    """Write flat state array back into body objects."""
    for k, b in enumerate(bodies):
        off = 13 * k
        b.p[:] = y[off:off+3]
        b.q[:] = y[off+3:off+7]
        b.v[:] = y[off+7:off+10]
        b.w[:] = y[off+10:off+13]

# Test round-trip
unpack_to_world(y0, bodies)
y_roundtrip = pack(bodies)
assert np.allclose(y0, y_roundtrip)
print("✓ pack → unpack → pack gives the same vector (round-trip works)")

---
# Step 3: The RHS Function — The heart of the IVP solver

The ODE solver calls a function `rhs(t, y)` that returns $\dot{y} = dy/dt$ — the **time derivative** of the state.

For each body, `ẏ` has 13 components:

| State | Derivative | Formula |
|-------|-----------|--------|
| position `p` (3) | velocity | $\dot{p} = v$ |
| quaternion `q` (4) | quat derivative | $\dot{q} = \frac{1}{2} q \otimes [\omega, 0]$ |
| velocity `v` (3) | linear acceleration | $\dot{v} = a_{\text{linear}}$ (from KKT) |
| spin `ω` (3) | angular acceleration | $\dot{\omega} = a_{\text{angular}}$ (from KKT) |

### The quaternion derivative

Quaternions represent 3D rotations as 4 numbers `[x, y, z, w]`. The derivative formula is:

$$\dot{q} = \frac{1}{2} \cdot q \otimes [\omega_x, \omega_y, \omega_z, 0]$$

where $\otimes$ is quaternion multiplication. You don't need to understand why — just know that this converts a spin rate `ω` into how the quaternion changes over time.

In [ ]:
def rhs_function(t, y, bodies, constraints, alpha, beta):
    """
    Compute dy/dt — the derivative of the full state vector.
    This is what scipy.solve_ivp calls at every internal step.
    """
    # 1. Write state from flat vector into body objects
    unpack_to_world(y, bodies)
    for b in bodies:
        b.q[:] = quat_normalize(b.q)  # keep quaternions unit-length
    
    # 2. Clear old forces and apply fresh ones
    for b in bodies:
        b.clear_forces()
        b.apply_force(b.mass * g)  # gravity
    
    # 3. Solve constrained dynamics (same as HybridSolver)
    Minv, J, F, rhs_v, _ = assemble_system(bodies, constraints, alpha, beta)
    a, _ = solve_kkt(Minv, J, F, rhs_v)
    
    # 4. Build the derivative vector ydot
    ydot = np.zeros_like(y)
    for i, b in enumerate(bodies):
        a_lin = a[6*i:6*i+3]     # linear acceleration from KKT
        a_ang = a[6*i+3:6*i+6]   # angular acceleration from KKT
        off = 13 * i
        
        ydot[off:off+3]    = b.v                          # ṗ = v
        ydot[off+3:off+7]  = quat_derivative(b.q, b.w)    # q̇ = 0.5 * q ⊗ [ω,0]
        ydot[off+7:off+10] = a_lin                        # v̇ = a_linear
        ydot[off+10:off+13] = a_ang                       # ω̇ = a_angular
    
    return ydot

# Test it
ydot_test = rhs_function(0.0, y0, bodies, [constraint], alpha, beta)

print("State derivative ẏ at t=0:")
for k, b in enumerate(bodies):
    off = 13 * k
    print(f"\n  {b.name}:")
    print(f"    ṗ (velocity):      {ydot_test[off:off+3]}")
    print(f"    q̇ (quat deriv):    {ydot_test[off+3:off+7]}")
    print(f"    v̇ (linear accel):  {ydot_test[off+7:off+10]}")
    print(f"    ω̇ (angular accel): {ydot_test[off+10:off+13]}")

---
# Step 4: Calling `scipy.solve_ivp`

`solve_ivp` takes our `rhs_function` and integrates it from `t=0` to `t=t_end`. It automatically chooses step sizes to meet the accuracy tolerances.

### Key parameters

| Parameter | Meaning |
|-----------|---------|
| `method='Radau'` | Implicit Runge-Kutta method, great for stiff problems |
| `rtol=1e-6` | Relative error tolerance (error / value < 10⁻⁶) |
| `atol=1e-8` | Absolute error tolerance (error < 10⁻⁸) |
| `max_step=1.0` | Never take a step larger than 1 second |

In [ ]:
from scipy.integrate import solve_ivp

# Reset bodies to initial state
anchor.p[:] = [0., 0., 2.]
anchor.v[:] = 0.; anchor.w[:] = 0.
payload.p[:] = [0., 0., 0.]
payload.v[:] = 0.; payload.w[:] = 0.

y0 = pack(bodies)
t_end = 2.0

sol = solve_ivp(
    fun=lambda t, y: rhs_function(t, y, bodies, [constraint], alpha, beta),
    t_span=(0.0, t_end),
    y0=y0,
    method='Radau',
    rtol=1e-6,
    atol=1e-8,
    max_step=1.0,
)

print(f"Integration result:")
print(f"  Success: {sol.success}")
print(f"  Number of time steps taken: {len(sol.t)}")
print(f"  Time range: {sol.t[0]:.3f} → {sol.t[-1]:.3f} s")
print(f"  State vector shape: {sol.y.shape}  (26 state vars × {len(sol.t)} time steps)")

## Let's look at the adaptive step sizes

In [ ]:
dt_steps = np.diff(sol.t)
print(f"Step sizes:")
print(f"  Smallest: {dt_steps.min():.6f} s")
print(f"  Largest:  {dt_steps.max():.6f} s")
print(f"  Mean:     {dt_steps.mean():.6f} s")
print(f"\nCompare: HybridSolver would need {int(t_end/0.01)} fixed steps of 0.01 s")
print(f"         IVP solver used only {len(sol.t)} steps (adaptive)!")

## Extract and plot trajectories

In [ ]:
import matplotlib.pyplot as plt

# Extract z-positions from the state vector
z_anchor  = sol.y[2, :]       # body 0, position z (index 2)
z_payload = sol.y[13 + 2, :]  # body 1, position z (index 15)

# Compute distance at each time step
distances = []
for k in range(len(sol.t)):
    p0 = sol.y[0:3, k]
    p1 = sol.y[13:16, k]
    distances.append(np.linalg.norm(p0 - p1))

fig, axes = plt.subplots(1, 3, figsize=(15, 4))

axes[0].plot(sol.t, z_anchor, label='Anchor z')
axes[0].plot(sol.t, z_payload, label='Payload z')
axes[0].set_xlabel('Time [s]'); axes[0].set_ylabel('Height [m]')
axes[0].set_title('Heights over time'); axes[0].legend(); axes[0].grid(True, alpha=0.3)

axes[1].plot(sol.t, distances, 'r-')
axes[1].axhline(2.0, color='k', ls='--', alpha=0.5, label='Required')
axes[1].set_xlabel('Time [s]'); axes[1].set_ylabel('Distance [m]')
axes[1].set_title('Constraint satisfaction'); axes[1].legend(); axes[1].grid(True, alpha=0.3)

axes[2].stem(sol.t[:-1], dt_steps, linefmt='b-', markerfmt='b.', basefmt=' ')
axes[2].set_xlabel('Time [s]'); axes[2].set_ylabel('Step size [s]')
axes[2].set_title('Adaptive step sizes'); axes[2].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print(f"\nFinal distance error: {abs(distances[-1] - 2.0):.2e} m")

---
# Step 5: Terminal Events — Stopping the simulation

The `HybridIVPSolver` can automatically **stop** when a condition is met (e.g., "payload hits the ground").

```python
def touchdown_event(t, y):
    z = y[13 * payload_index + 2]  # payload z-position
    return z - ground_z            # = 0 when z reaches ground

touchdown_event.terminal = True    # stop integration when event fires
touchdown_event.direction = -1.0   # only trigger when z is decreasing
```

`solve_ivp` monitors this function and stops the integration precisely when it crosses zero.

In [ ]:
# Example: stop when payload hits z = -10
ground_z = -10.0

def touchdown(t, y):
    return float(y[13 + 2] - ground_z)  # payload z - ground

touchdown.terminal = True
touchdown.direction = -1.0

# Reset
anchor.p[:] = [0., 0., 2.]; anchor.v[:] = 0.; anchor.w[:] = 0.
payload.p[:] = [0., 0., 0.]; payload.v[:] = 0.; payload.w[:] = 0.

sol2 = solve_ivp(
    fun=lambda t, y: rhs_function(t, y, bodies, [constraint], alpha, beta),
    t_span=(0.0, 100.0),  # long time horizon
    y0=pack(bodies),
    method='Radau', rtol=1e-6, atol=1e-8,
    events=touchdown,
)

print(f"Integration stopped at t = {sol2.t[-1]:.4f} s")
print(f"Payload z at stop: {sol2.y[15, -1]:.4f} m  (ground = {ground_z} m)")
if len(sol2.t_events[0]) > 0:
    print(f"Event triggered at t = {sol2.t_events[0][0]:.4f} s ✓")
else:
    print(f"Event not triggered (payload didn't reach ground).")

---
# Summary: HybridIVPSolver pipeline

```
Initial state y₀
       │
       ▼
┌──────────────────────────────────────┐
│  scipy.solve_ivp (Radau/BDF)         │
│                                      │
│  Repeatedly calls rhs(t, y):         │
│   ├─ Unpack y → body states          │
│   ├─ Clear & apply forces            │
│   ├─ assemble_system() → M⁻¹,J,F,rhs│
│   ├─ solve_kkt() → accelerations a   │
│   └─ Build ẏ = [v, q̇, a_lin, a_ang]  │
│                                      │
│  Automatically:                      │
│   ├─ Chooses step size for accuracy  │
│   ├─ Uses implicit methods (stable)  │
│   └─ Stops at terminal events        │
└──────────────────────────────────────┘
       │
       ▼
  Solution: y(t) at all time points
```

### Key differences from HybridSolver

1. **State packing**: All body states → one flat vector (scipy requirement)
2. **No manual integration**: scipy handles `v += a*dt`, `p += v*dt` internally
3. **Quaternion derivative**: Position uses velocity, but orientation needs the special $\dot{q}$ formula
4. **Adaptive stepping**: Fewer total evaluations for the same accuracy
5. **Terminal events**: Can stop automatically when conditions are met